In [ ]:
import os
from improc.io import parse_collection, DCAccessor
DCAccessor.register()
from holoviews import opts
import holoviews as hv

from inter_view.dashboards import SegmentationDashBoard, DashBoardCallback

hv.extension('bokeh', width=100)

# config

In [ ]:
basedir = '../data'

data_pattern = '{dataset_id}/{layer}/{fname_head}T{time:04d}.{ext}'
index = ['dataset_id', 'layer', 'time', 'fname_head']

nuclei_ch = 'Channel1-Deconv'
cell_ch = 'Channel0-Deconv'
nuclei_seg = 'nuclei_segmentation'
cell_seg = 'cell_segmentation'
lumen_seg = 'lumen_segmentation'

channel_config = {cell_ch:{'cmap':'red'},#, 'intensity_bounds':(1000,20000), 'slider_limits':(0,60000)},
                  nuclei_ch:{'cmap':'gray'},#, 'intensity_bounds':(300,15000), 'slider_limits':(0,60000)},
                  nuclei_seg:{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(0,2**16-1), 'bitdepth':16, 'opacity':0.2},
                  cell_seg:{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(0,2**16-1), 'bitdepth':16, 'opacity':0.2},
                  lumen_seg:{'cmap':'glasbey_hv_16bit', 'raster_aggregator':'first', 'intensity_bounds':(0,2**16-1), 'bitdepth':16, 'opacity':0.2}}

opts.defaults(opts.Image('channel.{}'.format(cell_seg), clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,256*256-1), tools=['hover']),
              opts.Image('channel.{}'.format(nuclei_seg), clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,256*256-1), tools=['hover']),
              opts.Image('channel.{}'.format(lumen_seg), clipping_colors={'min': (0, 0, 0, 0)}, clim=(1,256*256-1), tools=['hover']))

opts.defaults(opts.Image('channel', frame_width=1500))

In [ ]:
df = parse_collection(os.path.join(basedir, data_pattern), index)
df = df.dc[:,[nuclei_ch, cell_ch, nuclei_seg, cell_seg, lumen_seg]]
df = df.reset_index('fname_head')

df

# interactive dashboard

- ctrl + click to select the layers to overlay
- Use the export button to copy the currently loaded nuclei|lumen segmentation in a correction subfolder --> use the 3D_annotator notebook to correct the segmentation labels for retraining

In [ ]:
import panel as pn
import param
    
class SegmentationCallbackDashBoard(SegmentationDashBoard, DashBoardCallback):
    ''''''

    @param.depends('_complete_update_counter')
    def widgets(self):
        wg = [super().widgets(), self._export_widgets()]
        return pn.Column(*wg)

In [ ]:
import shutil
import numpy as np

def copy_nuclei_for_corrections(db):
    subdf = db.subdf.reset_index().set_index('layer').dc[nuclei_seg]
    
    subdf_out = subdf.copy().reset_index()
    subdf_out.layer = 'nuclei_corrections'
    
    # save labels as signed int16
    pred = subdf.dc.read()[0]
    subdf_out.dc.write(pred.astype(np.int16), compress=9, exist_ok=True)
copy_nuclei_for_corrections.name = 'copy nuclei for corrections'

def copy_lumen_for_corrections(db):
    subdf = db.subdf.reset_index().set_index('layer').dc[lumen_seg]
    
    subdf_out = subdf.copy().reset_index()
    subdf_out.layer = 'lumen_corrections'
    
    # save labels as signed int16
    pred = subdf.dc.read()[0]
    subdf_out.dc.write(pred.astype(np.int16), compress=9, exist_ok=True)
copy_lumen_for_corrections.name = 'copy lumen for corrections'

db = SegmentationCallbackDashBoard(df=df,
                                   multi_select_levels=['layer'],
                                   channel_config=channel_config,
                                   composite_channels=[nuclei_ch, cell_ch],
                                   overlay_channels=[nuclei_seg, cell_seg, lumen_seg],
                                   export_funs=[copy_nuclei_for_corrections,
                                                copy_lumen_for_corrections])

# disable cell and lumen segmentation by default
db.io_widgets()[1].value = [nuclei_ch, cell_ch, nuclei_seg]

db.panel().servable()